<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [0]:
!pip install distance
!pip install fuzzywuzzy

     |████████████████████████████████| 184kB 2.8MB/s 
  Created wheel for distance: filename=Distance-0.1.3-cp36-none-any.whl size=16261 sha256=0fe6ca449c6f982d57857f1b96ce279d438e9cae3e60e2525dcdfeaafb729ea2
  Stored in directory: /root/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e
Successfully built distance


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [0]:
# Load the CSV File created by the previous notebook
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_df_fe = '1hJJA8WXEFB9__8CzotLiaVAhb8owNXZ9'
id_train = '1CdbudO8YIOaVfnm4XrsS1pa28D_cHrXw'
id_nlp = '13Prkiot-joAo4E4uEqn0_HPpZCYfjqdS'
downloaded = drive.CreateFile({'id':id_train})
downloaded2 = drive.CreateFile({'id':id_df_fe})
downloaded3 = drive.CreateFile({'id':id_nlp})
downloaded.GetContentFile('train.csv')
downloaded2.GetContentFile('df_fe_without_preprocessing_train_quora.csv')
downloaded3.GetContentFile('nlp_features_train_quora.csv')

In [0]:
# avoid decoding problems
df = pd.read_csv("nlp_features_train_quora.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [0]:
df.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,4,1,51,88,8,13,4.0,20.0,0.200000,5,3,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154


In [0]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
#if os.path.isfile('nlp_features_train_quora.csv'):
    #dfnlp = pd.read_csv("nlp_features_train_quora.csv",encoding='latin-1')
#else:
    #print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train_quora.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train_quora.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [0]:
df1 = df.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
#df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
#df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
#df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [0]:
# Advanced NLP Features + Features from preprocessing
df1.head(3)

,id,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,4,1,51,88,8,13,4.0,20.0,0.200000,5,3,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,1,1,73,59,14,10,4.0,24.0,0.166667,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667


In [0]:
# Only Features present before preprocessing 
df2.head(3)

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0


In [0]:
print('Total Number of Features (NLP + Preproccessed) : ',df1.shape[1])
print('Total Number of Preprocessed Features : ',df2.shape[1])

Total Number of Features (NLP + Preproccessed) :  28
Total Number of Preprocessed Features :  12


**RANDOM SPLIT 70:30**

In [0]:
data = df.drop(['qid1','qid2','is_duplicate'],axis=1)
y_true = df['is_duplicate']

In [0]:
data.head(4)

,id,question1,question2,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,1,1,66,57,14,12,10.0,23.0,0.434783,2,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,4,1,51,88,8,13,4.0,20.0,0.200000,5,3,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,1,1,73,59,14,10,4.0,24.0,0.166667,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,why am i mentally very lonely how can i solve...,find the remainder when math 23 24 math i...,1,1,50,65,11,9,0.0,19.0,0.000000,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test, y_train, y_test = train_test_split(data, y_true, stratify=y_true, test_size=0.3)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


In [0]:
y_train.shape

(283003,)

In [0]:
#Applying on Train Data for Question 1
tfidf = TfidfVectorizer(lowercase=False, min_df=10)
D_Train_q1 = tfidf.fit_transform(X_train['question1'])

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [0]:
#Applying on Test Data for Question 1

D_Test_q1 = tfidf.transform(X_test['question1'])

In [0]:
print('Shape of Training Data',D_Train_q1.shape)
print('Shape of Testing Data',D_Test_q1.shape)

Shape of Training Data (283003, 12313)
Shape of Testing Data (121287, 12313)


In [0]:
print(tfidf.get_feature_names()[:10])
print(word2tfidf)

['00', '01', '04', '06', '09', '10', '100', '100k', '100m', '101']
{'00': 10.508693873077682, '01': 10.608777331634665, '04': 9.997868249311692, '06': 10.845166109698894, '09': 10.988266953339568, '10': 6.273897475386485, '100': 7.320768294250582, '100k': 8.988868119333269, '100m': 11.155321038002734, '101': 10.988266953339568, '1060': 11.155321038002734, '1070': 11.068309661013105, '1080p': 10.417722094871955, '1099': 11.155321038002734, '10k': 8.275101651570587, '10kg': 10.914158981185846, '10m': 11.068309661013105, '10th': 8.583403011225105, '11': 7.742075317824405, '110': 10.37516248045316, '110v': 11.068309661013105, '11th': 9.171189676127224, '12': 7.383605578309649, '120': 9.88965466467146, '1200': 11.155321038002734, '125': 10.462173857442789, '12k': 9.746553821030785, '12th': 7.984871807040008, '13': 7.904242072639899, '130': 10.72000296674489, '1300d': 11.155321038002734, '135': 10.914158981185846, '13k': 11.068309661013105, '13th': 10.780627588561323, '14': 8.031755392938859

In [0]:
#Applying on Train Data for Question 2
tfidf_q2 = TfidfVectorizer(lowercase=False, min_df=10)
D_Train_q2 = tfidf_q2.fit_transform(X_train['question2'])

# dict key:word and value:tf-idf score
word2tfidf_q2 = dict(zip(tfidf_q2.get_feature_names(), tfidf_q2.idf_))

In [0]:
#Applying on Test Data for Question 1

D_Test_q2 = tfidf_q2.transform(X_test['question2'])

In [0]:
print('Shape of Training Data',D_Train_q2.shape)
print('Shape of Testing Data',D_Test_q2.shape)

Shape of Training Data (283003, 11862)
Shape of Testing Data (121287, 11862)


In [0]:
import scipy as sp
D_Train = sp.sparse.hstack((D_Train_q1,D_Train_q2))
D_Test= sp.sparse.hstack((D_Test_q1,D_Test_q2))

In [0]:
print(D_Train.shape)
print(D_Test.shape)

(283003, 24175)
(121287, 24175)


In [0]:
X_train11 = sp.sparse.hstack((X_train[['cwc_min','cwc_max','csc_min','csc_max','ctc_min','ctc_max','last_word_eq','first_word_eq','abs_len_diff',
                                             'mean_len','token_set_ratio','token_sort_ratio','fuzz_ratio','fuzz_partial_ratio','longest_substr_ratio','freq_qid1',
                                             'freq_qid2','q1len','q2len','q1_n_words','q2_n_words','word_Common','word_Total',
                                              'word_share','freq_q1+q2','freq_q1-q2']],D_Train), format='csc')

In [0]:
print(X_train11.shape)

(283003, 24201)


In [0]:
X_cols = X_train[['cwc_min','cwc_max','csc_min','csc_max','ctc_min','ctc_max','last_word_eq','first_word_eq','abs_len_diff',
                                             'mean_len','token_set_ratio','token_sort_ratio','fuzz_ratio','fuzz_partial_ratio','longest_substr_ratio','freq_qid1',
                                             'freq_qid2','q1len','q2len','q1_n_words','q2_n_words','word_Common','word_Total',
                                              'word_share','freq_q1+q2','freq_q1-q2']].columns.tolist()+tfidf.get_feature_names()+tfidf_q2.get_feature_names()
X_cols

['cwc_min',
 'cwc_max',
 'csc_min',
 'csc_max',
 'ctc_min',
 'ctc_max',
 'last_word_eq',
 'first_word_eq',
 'abs_len_diff',
 'mean_len',
 'token_set_ratio',
 'token_sort_ratio',
 'fuzz_ratio',
 'fuzz_partial_ratio',
 'longest_substr_ratio',
 'freq_qid1',
 'freq_qid2',
 'q1len',
 'q2len',
 'q1_n_words',
 'q2_n_words',
 'word_Common',
 'word_Total',
 'word_share',
 'freq_q1+q2',
 'freq_q1-q2',
 '00',
 '01',
 '04',
 '06',
 '09',
 '10',
 '100',
 '100k',
 '100m',
 '101',
 '1060',
 '1070',
 '1080p',
 '1099',
 '10k',
 '10kg',
 '10m',
 '10th',
 '11',
 '110',
 '110v',
 '11th',
 '12',
 '120',
 '1200',
 '125',
 '12k',
 '12th',
 '13',
 '130',
 '1300d',
 '135',
 '13k',
 '13th',
 '14',
 '140',
 '14k',
 '14th',
 '15',
 '150',
 '1500',
 '150k',
 '151',
 '155',
 '15k',
 '15th',
 '16',
 '160',
 '1600',
 '1611',
 '16gb',
 '16th',
 '17',
 '1700',
 '1746',
 '17th',
 '18',
 '180',
 '1800',
 '1812',
 '18k',
 '18th',
 '19',
 '190',
 '1922',
 '1938',
 '1947',
 '1952',
 '1957',
 '1960',
 '1960s',
 '1962',
 '196

In [0]:
print(len(X_cols))

24201


In [0]:
X_test11 = sp.sparse.hstack((X_test[['cwc_min','cwc_max','csc_min','csc_max','ctc_min','ctc_max','last_word_eq','first_word_eq','abs_len_diff',
                                             'mean_len','token_set_ratio','token_sort_ratio','fuzz_ratio','fuzz_partial_ratio','longest_substr_ratio','freq_qid1',
                                             'freq_qid2','q1len','q2len','q1_n_words','q2_n_words','word_Common','word_Total',
                                              'word_share','freq_q1+q2','freq_q1-q2']],D_Test), format='csc')

In [0]:
X_cols = X_train[['cwc_min','cwc_max','csc_min','csc_max','ctc_min','ctc_max','last_word_eq','first_word_eq','abs_len_diff',
                                             'mean_len','token_set_ratio','token_sort_ratio','fuzz_ratio','fuzz_partial_ratio','longest_substr_ratio','freq_qid1',
                                             'freq_qid2','q1len','q2len','q1_n_words','q2_n_words','word_Common','word_Total',
                                              'word_share','freq_q1+q2','freq_q1-q2']].columns.tolist()+tfidf.get_feature_names()+tfidf_q2.get_feature_names()
print(len(X_cols))

24201


In [0]:
import scipy.sparse

In [0]:
X_test11.shape

(121287, 24201)

In [0]:
#scipy.sparse.save_npz("final_features_train11.npz", X_train11)
scipy.sparse.save_npz("final_features_test11.npz", X_test11)
#np.savez("final_features_train1.npz", data = X_train1.data, indices = X_train1.indices, indptr = X_train1.indptr, shape = X_train1.shape)

In [0]:
# storing the final features to csv file -- Normal Tfidf Features + NLP + Preprocessing
if not os.path.isfile('final_features_train11.npz'):
    scipy.sparse.save_npz("final_features_train11.npz", X_train11)

In [0]:
from google.colab import drive
drive.mount('drive')
!cp final_features_train11.npz drive/My\ Drive/Quora\ Question\ Pair\ Similarity/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
# storing the final features to csv file -- Normal Tfidf Features + NLP + Preprocessing
if not os.path.isfile('final_features_test11.npz'):
    scipy.sparse.save_npz("final_features_test11.npz", X_test11)

In [0]:
!cp final_features_test11.npz drive/My\ Drive/Quora\ Question\ Pair\ Similarity/

In [0]:
y_train.to_csv('Y_tfidf_train1.csv')
y_test.to_csv('Y_tfidf_test1.csv')

In [0]:
print(X_train.shape)
print(X_train11.shape)
y_train.head(5)

(283003, 29)
(283003, 24201)


378640    1
53161     0
403409    0
73881     1
217871    0
Name: is_duplicate, dtype: int64

In [0]:
if not os.path.isfile('Y_tfidf_train.csv'):
    y_train.to_csv('Y_tfidf_train.csv')

In [0]:
!cp Y_tfidf_train1.csv drive/My\ Drive/Quora\ Question\ Pair\ Similarity/

In [0]:
if not os.path.isfile('Y_tfidf_test.csv'):
    y_test.to_csv('Y_tfidf_test.csv')

In [0]:
!cp Y_tfidf_test1.csv drive/My\ Drive/Quora\ Question\ Pair\ Similarity/

***BUILDING THE GLOVE VECTOR FOR XGBOOST MODEL : ***

In [0]:
# merge texts
questions = list(X_train['question1']) + list(X_train['question2'])
questions_test = list(X_test['question1']) + list(X_test['question2'])

tfidf_w2v = TfidfVectorizer(lowercase=False, )
tfidf_w2v.fit_transform(questions)
tfidf_w2v.transform(questions_test)

# dict key:word and value:tf-idf score -- Train
word2v_tfidf = dict(zip(tfidf_w2v.get_feature_names(), tfidf_w2v.idf_))


- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [0]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_train['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2v_tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
X_train['q1_feats_m'] = list(vecs1)


100%|██████████| 283003/283003 [50:29<00:00, 93.41it/s]


In [0]:
vecs2 = []
for qu2 in tqdm(list(X_train['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2v_tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
X_train['q2_feats_m'] = list(vecs2)

100%|██████████| 283003/283003 [46:25<00:00, 102.44it/s]


In [0]:
vecs3 = []
# tqdm is used to print the progress bar
for qu1 in tqdm(list(X_test['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec3 = word1.vector
        # fetch df score
        try:
            idf = word2v_tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec3 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs3.append(mean_vec1)
X_test['q1_feats_m'] = list(vecs3)

100%|██████████| 121287/121287 [19:41<00:00, 102.68it/s]


In [0]:
vecs2 = []
for qu2 in tqdm(list(X_test['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2v_tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
X_test['q2_feats_m'] = list(vecs2)

100%|██████████| 121287/121287 [20:06<00:00, 100.52it/s]


In [0]:
df3 = X_train.drop(['question1','question2'],axis=1)
df3_q1 = pd.DataFrame(X_train.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(X_train.q2_feats_m.values.tolist(), index= df3.index)

In [0]:
df3_q1.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
82817,7.090451,-1.862659,-49.456611,-56.283810,-51.790498,63.632962,-5.295274,33.157303,2.342729,-2.263824,-20.906344,-29.466421,-16.201112,-31.802540,26.538220,-25.693567,-13.743724,17.648643,1.335896,11.151436,-16.760023,-17.856441,45.862753,37.486609,66.325921,17.413087,19.395941,29.618106,13.226200,17.091385,11.305418,-4.288049,-47.423933,-14.484315,30.578126,-8.155957,14.532606,51.160779,-6.908237,-22.707035,...,36.650244,-0.115224,-43.242468,52.987339,-12.279259,-22.431540,16.724183,10.282804,11.515793,21.161873,17.279759,50.181452,-24.929224,-0.228589,37.060885,-27.933186,12.923378,4.830302,-24.380740,16.536875,-24.596308,-17.171700,-1.818064,-13.009831,-5.159232,24.839301,-59.064038,16.959859,25.912318,-27.372709,17.720954,-57.546169,1.267223,-36.173370,-16.198098,18.803975,33.896629,-5.013838,-45.741878,10.515905
46503,236.278434,-12.043294,-155.940614,-203.949228,2.923514,185.491072,27.326979,28.423408,-20.265133,221.221346,-106.665475,130.085091,-282.444204,-156.905651,-26.261436,-157.004356,93.453060,59.612922,135.692605,124.282525,-138.247019,-62.208067,222.845760,191.486206,196.075822,183.210677,121.085743,-95.832845,-103.076777,218.375359,11.247810,-96.231001,-157.886526,3.273651,230.757905,106.250499,-24.235885,202.287698,15.254596,-6.444737,...,-22.546498,121.885950,-67.387534,5.815701,-196.880971,-114.471981,128.953266,-66.178369,-103.455027,139.348995,138.113370,2.299359,-191.830786,89.618216,294.395837,-208.878317,67.115818,171.569718,-0.617993,51.973400,-33.301939,-132.065082,-81.283939,-168.020953,-100.859046,-80.655482,-115.326560,75.787692,103.733318,-100.768583,123.769042,-142.562985,94.877666,-54.925222,-39.890438,-40.212795,188.642079,9.389171,-211.766986,106.415785


In [0]:
df3_q2.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
82817,9.771384,-58.417895,-43.704746,-62.829259,-36.794363,30.516895,-32.711100,21.998377,-12.436128,17.559398,-4.343313,-18.122972,-20.770929,-34.646296,39.295530,-35.672146,-22.946995,-8.553684,14.142848,-1.865847,-50.958218,-63.952105,60.541312,43.759680,28.362443,-11.124834,47.483066,-2.231828,-1.589954,64.452450,5.951896,-0.309039,-53.138069,32.941900,38.176794,6.399209,11.852434,76.761377,3.952522,-15.839295,...,38.776590,-33.523294,-39.500026,39.272362,-50.031311,8.778709,45.040307,-52.857321,21.911999,19.666619,38.122326,75.046336,-12.856911,-29.398831,34.125122,-45.786491,-0.182641,76.021515,-31.316083,8.629522,-27.158724,-23.636512,-9.718953,-9.690776,4.183228,28.652466,-65.830930,54.284922,16.298943,-57.916700,33.691885,-47.942557,29.853445,2.017321,-5.660319,-14.542119,29.515649,-5.933426,-8.200077,41.193522
46503,88.925995,-101.541529,-131.319210,-214.806647,-11.660673,171.403547,199.212222,104.993217,-104.814489,188.182673,30.730717,210.877041,-279.974276,-69.387151,-15.623517,20.556046,84.840928,61.991193,266.679691,281.397555,-138.021350,-137.069819,224.920750,326.763198,141.608034,223.962299,214.501469,-168.327790,4.398235,320.885772,-4.342189,-113.702238,-111.987619,-131.740259,160.994652,152.428696,86.029221,113.740175,-188.464391,118.159570,...,-68.270452,100.705458,-224.140924,-18.057727,-104.662768,-198.099155,-35.269101,-98.383430,-217.400657,72.699174,149.524094,-47.302379,-249.560254,108.584419,248.776134,-143.598504,155.502118,163.948656,171.100388,40.100747,50.388014,-129.536353,-82.574858,-162.977166,-200.453017,-107.973780,-140.235583,66.276908,43.240257,-71.820817,189.487212,-169.385001,125.681656,-43.487759,-32.775444,-22.902526,23.071059,34.148737,-180.392371,206.077933


In [0]:
X_train.head(2)

,id,question1,question2,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,q1_feats_m,q2_feats_m
82817,82817,should i have a girlfriend or not,why should i have a girlfriend boyfriend,4,2,34,43,7,8,5.0,15.0,0.333333,6,2,0.999900,0.499975,0.799984,0.666656,0.714276,0.714276,0.0,0.0,0.0,7.0,88,71,80,85,0.771429,"[7.090451002120972, -1.8626587390899658, -49.4...","[9.77138376235962, -58.41789501160383, -43.704..."
46503,46503,i have got 19395 rank in the jee advance and 4...,my obc ncl rank in jee advanced is 3014 what ...,1,1,87,105,19,19,4.0,35.0,0.114286,2,0,0.399996,0.399996,0.333328,0.222220,0.315788,0.285713,0.0,0.0,2.0,20.0,65,66,51,54,0.181818,"[236.27843356132507, -12.043294429779053, -155...","[88.92599520087242, -101.54152899980545, -131...."


In [0]:
df3_q1.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
82817,7.090451,-1.862659,-49.456611,-56.283810,-51.790498,63.632962,-5.295274,33.157303,2.342729,-2.263824,-20.906344,-29.466421,-16.201112,-31.802540,26.538220,-25.693567,-13.743724,17.648643,1.335896,11.151436,-16.760023,-17.856441,45.862753,37.486609,66.325921,17.413087,19.395941,29.618106,13.226200,17.091385,11.305418,-4.288049,-47.423933,-14.484315,30.578126,-8.155957,14.532606,51.160779,-6.908237,-22.707035,...,36.650244,-0.115224,-43.242468,52.987339,-12.279259,-22.431540,16.724183,10.282804,11.515793,21.161873,17.279759,50.181452,-24.929224,-0.228589,37.060885,-27.933186,12.923378,4.830302,-24.380740,16.536875,-24.596308,-17.171700,-1.818064,-13.009831,-5.159232,24.839301,-59.064038,16.959859,25.912318,-27.372709,17.720954,-57.546169,1.267223,-36.173370,-16.198098,18.803975,33.896629,-5.013838,-45.741878,10.515905
46503,236.278434,-12.043294,-155.940614,-203.949228,2.923514,185.491072,27.326979,28.423408,-20.265133,221.221346,-106.665475,130.085091,-282.444204,-156.905651,-26.261436,-157.004356,93.453060,59.612922,135.692605,124.282525,-138.247019,-62.208067,222.845760,191.486206,196.075822,183.210677,121.085743,-95.832845,-103.076777,218.375359,11.247810,-96.231001,-157.886526,3.273651,230.757905,106.250499,-24.235885,202.287698,15.254596,-6.444737,...,-22.546498,121.885950,-67.387534,5.815701,-196.880971,-114.471981,128.953266,-66.178369,-103.455027,139.348995,138.113370,2.299359,-191.830786,89.618216,294.395837,-208.878317,67.115818,171.569718,-0.617993,51.973400,-33.301939,-132.065082,-81.283939,-168.020953,-100.859046,-80.655482,-115.326560,75.787692,103.733318,-100.768583,123.769042,-142.562985,94.877666,-54.925222,-39.890438,-40.212795,188.642079,9.389171,-211.766986,106.415785


In [0]:
if not os.path.isfile('final_features_quora_w2v_train1.csv'):
    df3_q1['id']=X_train['id']
    df3_q2['id']=X_train['id']
    df1  = X_train.drop(['question1','question2','q1_feats_m','q2_feats_m'],axis=1) #This has all NLP + Preprocess features -- Test
    df2  = df3_q1.merge(df3_q2, on='id',how='left') #This has all the tfidf_w2v features -- Test
    result1  = df1.merge(df2, on='id',how='left')
    result1.to_csv('final_features_quora_w2v_train1.csv') #All Test features (NLP + Preproces + W2V)

In [0]:
result1.head(2)

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,is_duplicate,is_duplicate,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,...,56_y,57_y,58_y,59_y,60_y,61_y,62_y,63_y,64_y,65_y,66_y,67_y,68_y,69_y,70_y,71_y,72_y,73_y,74_y,75_y,76_y,77_y,78_y,79_y,80_y,81_y,82_y,83_y,84_y,85_y,86_y,87_y,88_y,89_y,90_y,91_y,92_y,93_y,94_y,95_y
0,82817,4,2,34,43,7,8,5.0,15.0,0.333333,6,2,0.999900,0.499975,0.799984,0.666656,0.714276,0.714276,0.0,0.0,0.0,7.0,88,71,80,85,0.771429,0,0,7.090451,-1.862659,-49.456611,-56.283810,-51.790498,63.632962,-5.295274,33.157303,2.342729,-2.263824,-20.906344,...,38.776590,-33.523294,-39.500026,39.272362,-50.031311,8.778709,45.040307,-52.857321,21.911999,19.666619,38.122326,75.046336,-12.856911,-29.398831,34.125122,-45.786491,-0.182641,76.021515,-31.316083,8.629522,-27.158724,-23.636512,-9.718953,-9.690776,4.183228,28.652466,-65.830930,54.284922,16.298943,-57.916700,33.691885,-47.942557,29.853445,2.017321,-5.660319,-14.542119,29.515649,-5.933426,-8.200077,41.193522
1,46503,1,1,87,105,19,19,4.0,35.0,0.114286,2,0,0.399996,0.399996,0.333328,0.222220,0.315788,0.285713,0.0,0.0,2.0,20.0,65,66,51,54,0.181818,0,0,236.278434,-12.043294,-155.940614,-203.949228,2.923514,185.491072,27.326979,28.423408,-20.265133,221.221346,-106.665475,...,-68.270452,100.705458,-224.140924,-18.057727,-104.662768,-198.099155,-35.269101,-98.383430,-217.400657,72.699174,149.524094,-47.302379,-249.560254,108.584419,248.776134,-143.598504,155.502118,163.948656,171.100388,40.100747,50.388014,-129.536353,-82.574858,-162.977166,-200.453017,-107.973780,-140.235583,66.276908,43.240257,-71.820817,189.487212,-169.385001,125.681656,-43.487759,-32.775444,-22.902526,23.071059,34.148737,-180.392371,206.077933


In [0]:
#from google.colab import drive
#.mount('drive')
!cp final_features_quora_w2v_train1.csv drive/My\ Drive/Quora\ Question\ Pair\ Similarity/

In [0]:
#TEST
df3_test = X_test.drop(['question1','question2'],axis=1)
df3_q1_test = pd.DataFrame(X_test.q1_feats_m.values.tolist(), index= df3_test.index)
df3_q2_test = pd.DataFrame(X_test.q2_feats_m.values.tolist(), index= df3_test.index)

In [0]:
df1.head(2)

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,is_duplicate
367687,367687,1,1,49,70,11,15,5.0,22.0,0.227273,2,0,0.799984,0.799984,0.599988,0.428565,0.636358,0.466664,0.0,0.0,4.0,13.0,78,69,55,63,0.340000,0
323231,323231,1,1,169,45,27,8,4.0,32.0,0.125000,2,0,0.249994,0.071428,0.749981,0.299997,0.499994,0.148148,0.0,0.0,19.0,17.5,67,37,30,67,0.217391,0


In [0]:
df3_q1_test.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
367687,115.762490,-74.208016,-55.801146,-101.851861,3.075431,-14.421755,42.387522,-22.599854,-21.790135,115.961914,1.999526,45.685042,-84.461707,-29.853370,-8.588731,-65.932558,72.800996,104.675919,182.562789,148.299775,10.354449,-74.611714,12.797295,149.927705,69.846804,85.770444,55.160944,-16.784901,126.761665,123.049324,104.140384,-53.214671,-64.514896,-81.122351,48.280865,48.775062,7.854183,61.191469,-75.663131,73.877873,...,-39.891198,-20.559678,-88.207386,-20.554181,-31.601122,-156.366639,57.618311,-33.865563,-62.502476,113.107333,32.744692,-14.193596,-50.350885,48.369219,33.693817,-86.268487,94.232271,18.921725,-32.059882,67.115843,50.816629,-69.235400,-56.765374,-148.404638,-36.795980,-49.558906,-28.930060,14.620721,58.956471,-19.239242,25.967630,-35.463388,47.465074,-51.232167,-17.384566,53.438887,-86.522493,38.680409,-102.000295,-4.071816
323231,-149.229956,-56.839321,-266.568144,-357.284265,76.632574,242.508771,497.806480,111.412513,69.693705,5.337946,-105.328928,-4.362133,-165.641075,-269.083467,-176.609309,161.307403,121.178241,131.720151,-52.664246,114.315917,64.198368,-117.495935,157.187216,319.458462,138.909716,77.507986,191.199561,22.978110,33.137756,494.803115,78.712156,55.009245,-278.688182,95.958933,278.746048,204.621277,17.699216,-16.770972,-61.461132,4.542339,...,-115.798438,-80.152747,-274.678704,162.912881,-273.506435,61.476683,52.363004,-191.135542,-53.587091,-36.241436,138.896422,-17.472535,-355.587407,207.532758,-16.513600,-293.383927,-67.644911,229.642821,205.975175,156.490455,50.559245,-21.168574,53.345642,21.526252,-48.332951,-285.618742,-217.687694,49.349468,-132.529360,-138.531087,507.971029,5.471333,108.820714,27.968117,-103.144020,103.202134,-201.490729,39.495133,-154.603075,336.167660


In [0]:
# storing the final features to csv file
if not os.path.isfile('final_features_quora_w2v_test.csv'):
    df3_q1_test['id']=X_test['id']
    df3_q2_test['id']=X_test['id']
    df1  = X_test.drop(['question1','question2','q1_feats_m','q2_feats_m'],axis=1) #This has all NLP + Preprocess features -- Test
    df2  = df3_q1_test.merge(df3_q2_test, on='id',how='left') #This has all the tfidf_w2v features -- Test
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features_quora_w2v_test.csv') #All Test features (NLP + Preproces + W2V)

In [0]:
!cp final_features_quora_w2v_test.csv drive/My\ Drive/Quora\ Question\ Pair\ Similarity/

In [0]:
X_train = pd.concat([X_train,y_train],axis = 1)
X_train.head(2)

,id,question1,question2,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,q1_feats_m,q2_feats_m,is_duplicate,is_duplicate
82817,82817,should i have a girlfriend or not,why should i have a girlfriend boyfriend,4,2,34,43,7,8,5.0,15.0,0.333333,6,2,0.999900,0.499975,0.799984,0.666656,0.714276,0.714276,0.0,0.0,0.0,7.0,88,71,80,85,0.771429,"[7.090451002120972, -1.8626587390899658, -49.4...","[9.77138376235962, -58.41789501160383, -43.704...",0,0
46503,46503,i have got 19395 rank in the jee advance and 4...,my obc ncl rank in jee advanced is 3014 what ...,1,1,87,105,19,19,4.0,35.0,0.114286,2,0,0.399996,0.399996,0.333328,0.222220,0.315788,0.285713,0.0,0.0,2.0,20.0,65,66,51,54,0.181818,"[236.27843356132507, -12.043294429779053, -155...","[88.92599520087242, -101.54152899980545, -131....",0,0


In [0]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 28
Number of features in question1 w2v  dataframe : 97
Number of features in question2 w2v  dataframe : 97
Number of features in final dataframe  : 222


In [0]:
result.head(2)

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,is_duplicate,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,...,56_y,57_y,58_y,59_y,60_y,61_y,62_y,63_y,64_y,65_y,66_y,67_y,68_y,69_y,70_y,71_y,72_y,73_y,74_y,75_y,76_y,77_y,78_y,79_y,80_y,81_y,82_y,83_y,84_y,85_y,86_y,87_y,88_y,89_y,90_y,91_y,92_y,93_y,94_y,95_y
0,367687,1,1,49,70,11,15,5.0,22.0,0.227273,2,0,0.799984,0.799984,0.599988,0.428565,0.636358,0.466664,0.0,0.0,4.0,13.0,78,69,55,63,0.340000,0,115.762490,-74.208016,-55.801146,-101.851861,3.075431,-14.421755,42.387522,-22.599854,-21.790135,115.961914,1.999526,45.685042,...,-42.050336,-33.474887,-128.849563,-34.446405,-53.191685,-189.758148,117.798423,-27.428825,-140.840117,111.528942,30.514188,-10.811285,-46.445400,64.471699,43.933532,-121.950046,148.121045,57.409255,-50.875158,107.591816,108.852331,-117.175637,-129.925104,-208.897631,-90.023910,-59.831604,-39.348337,8.588068,149.002959,-20.625835,49.385490,-32.689386,58.150453,-66.84766,-62.089226,122.674473,-48.233593,43.498084,-111.942048,53.880565
1,323231,1,1,169,45,27,8,4.0,32.0,0.125000,2,0,0.249994,0.071428,0.749981,0.299997,0.499994,0.148148,0.0,0.0,19.0,17.5,67,37,30,67,0.217391,0,-149.229956,-56.839321,-266.568144,-357.284265,76.632574,242.508771,497.806480,111.412513,69.693705,5.337946,-105.328928,-4.362133,...,17.451739,-62.360937,-61.133815,124.617794,-35.317108,4.815766,142.150825,-91.316613,42.062603,44.234816,13.178004,66.292820,-12.814468,132.130166,92.724102,-110.298620,-32.647367,93.403845,32.270570,68.259275,-19.190544,-56.931385,-27.851123,-61.624805,-0.497546,-66.170128,-137.299830,61.841954,27.854008,-12.460336,72.404718,-53.841319,69.895342,19.37914,0.380617,63.147189,-15.601520,-39.969135,-42.379380,80.420763
